### Download and Install Dependencies

In [ ]:
!wget https://raw.githubusercontent.com/xenova/transformers.js/main/scripts/requirements.txt
!wget https://raw.githubusercontent.com/xenova/transformers.js/main/scripts/convert.py
!pip install -r requirements.txt -q
!pip install huggingface_hub -q

In [ ]:
import os
from huggingface_hub import HfApi, notebook_login, whoami, create_repo, ModelCard, ModelCardData

### Login to 🤗 Hub
Make sure you add HF_TOKEN to your notebook secrets

In [ ]:
notebook_login()
user = whoami()['name']
api = HfApi()
print(f"Logged in to 🤗 as {user}")

### Setup Model Credentials
Pick a model from the  [🤗 Hub](https://huggingface.co/models).  
Set your username on the 🤗 Hub to deploy the converted model.

In [ ]:
model = "textattack/bert-base-uncased-rotten-tomatoes" # @param {type:"string"}
task = "text-classification" # @param {type:"string"}
user = "codewithkyrian" # @param {type:"string"}

### Convert the model to ONNX
This might take a moment.

In [ ]:
os.makedirs("models", exist_ok=True)
!python convert.py --quantize --model_id $model --task $task

### Upload Converted model to 🤗 Hub

In [ ]:
model_name = model.split("/")[-1]
repo_id = f"{user}/{model_name}" # new repo id with user
print(f"Repo ID: {repo_id}")

In [ ]:
# Create the repo
repo_url = create_repo(repo_id, exist_ok=True)

In [ ]:
# upload the model to the hub
api.upload_folder(
    folder_path=f"models/{model}", # default output path from convert.py
    repo_id=repo_id,
)

In [ ]:
card = ModelCard.load(model) # get the old model card
card_meta = card.data.to_dict()
card_meta['library_name'] = "Transformers PHP"
card_meta.setdefault('tags', []) # sometimes, tags key doesn't exist
card_meta['tags'] += ["onnx"]
card_meta['pipeline_tag'] = task
card_meta = ModelCardData(**card_meta)

In [ ]:
# Add transformers.js modelcard template to existing model card
content = f"""
---
{card_meta.to_yaml()}
---

https://huggingface.co/{model} with ONNX weights to be compatible with Transformers PHP

{card.text}
---

Note: Having a separate repo for ONNX weights is intended to be a temporary solution until ONNXRuntime gains more traction. If you would like to make your models web-ready, we recommend converting to ONNX using [🤗 Optimum](https://huggingface.co/docs/optimum/index) and structuring your repo like this one (with ONNX weights located in a subfolder named `onnx`).
"""

In [ ]:
ModelCard(content).push_to_hub(repo_id) # push the new model card to the hub